<a href="https://colab.research.google.com/github/Funkyojo/FishPose/blob/master/FishPose_base.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os
import math
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.python.framework import ops
from PIL import Image, ImageDraw
import sys
import random

In [0]:
LBL_NUM = 13
LBL_W = 3
SL_WEIGHT = 50 # weight of area between line cost

class FP_base(keras.Model):
    def __init__(self):
        super(FP_base, self).__init__()
        self.c1 = keras.layers.Conv2D(filters=48, kernel_size=5, strides=2, activation=tf.nn.relu, padding='same', input_shape=(100, 300, 1))
        self.p1 = keras.layers.MaxPool2D(pool_size = 2, padding = 'same')

        self.c2 = keras.layers.Conv2D(filters=128, kernel_size=5, strides=1, activation=tf.nn.relu, padding='same')
        self.p2 = keras.layers.MaxPool2D(pool_size = 2, padding = 'same')
        
        self.c3 = keras.layers.Conv2D(filters=192, kernel_size=5, strides=2, activation=tf.nn.relu, padding='same')
        self.p3 = keras.layers.MaxPool2D(pool_size = 2, padding = 'same')

        self.c4 = keras.layers.Conv2D(filters=64, kernel_size=1, strides=1, activation=tf.nn.relu, padding='same')

        self.f = keras.layers.Flatten()
        
        self.d7 = keras.layers.Dense(512, activation=tf.nn.relu)
        self.d8 = keras.layers.Dense(LBL_NUM*LBL_W, activation= None)        

    def call(self, inputs):
        X = self.c1(inputs)
        X = self.p1(X)
        X = self.c2(X)
        X = self.p2(X)
        X = self.c3(X)
        X = self.p3(X)
        X = self.c4(X)
        X = self.f(X)
        X = self.d7(X)
        X = self.d8(X)
        return X

def read_label(f_name):
    """
    Read a "FishPose" label in the form: pres y   x
    """
    f = open(f_name)
    label = np.zeros((LBL_NUM, LBL_W))
    l_counter = 0
    for line in f:
        things = line.strip().split('\t')
        label[l_counter,:] = list(map(float, things))
        l_counter += 1
    f.close()
    return label

def load_dataset(folder):
    fil_list = os.listdir(folder)
    n_f = len(fil_list)//2
    X_orig = np.zeros((n_f, 100, 300, 1))
    Y_orig = np.zeros((n_f, LBL_NUM, LBL_W))
    counter = 0
    for f in fil_list:
        if f.endswith(".txt"):
            img_f = f[:-3] + "bmp"
            X_orig[counter,:,:,0] = np.array(Image.open(folder+img_f))
            Y_orig[counter,:,:] = read_label(folder+f) 
            counter += 1
    X = X_orig/255.
    return X, Y_orig

def FP_loss_Pres(y_true, y_pred):
    pres_true = y_true[:,:,0]
    pred = tf.reshape(y_pred, (-1, LBL_NUM, LBL_W))
    pres_pred = pred[:,:,0]
    pres_cost = tf.nn.sigmoid_cross_entropy_with_logits(labels = pres_true, logits = pres_pred)

    pres_not_ignore = tf.math.sign(pres_true + 1)
    pres_inds = tf.nn.relu(pres_true)

    RealPresCost = tf.math.reduce_mean(tf.math.multiply(pres_not_ignore, pres_cost))
    return RealPresCost

def FP_loss_Coord(y_true, y_pred):
    pres_true = y_true[:,:,0]
    coord_true = y_true[:,:,1:]
    pred = tf.reshape(y_pred, (-1, LBL_NUM, LBL_W))

    coord_pred = pred[:,:,1:]
    coord_cost = tf.norm(tf.multiply(coord_pred - coord_true,[1,3]), ord = 2, axis=2)

    pres_inds = tf.nn.relu(pres_true)
    RealCoordCost = tf.math.reduce_mean(tf.math.multiply(pres_inds, coord_cost))
    return RealCoordCost

def FP_loss_SL(y_true, y_pred):
    pres_true = y_true[:,:,0]
    coord_true = y_true[:,:,1:]
    pred = tf.reshape(y_pred, (-1, LBL_NUM, LBL_W))

    pres_pred = pred[:,:,0]
    coord_pred = pred[:,:,1:]

    pres_inds = tf.nn.relu(pres_true)

    return line_loss(pres_inds, coord_true, coord_pred)


def shoelace(LL):
    """
    Input shape: (m, len(lines_care), 4, 2);
    3rd dim: [true_b, true_t, pred_b, pred_t], 4th dim:[Y, X]
    return shape: (m, len(lines_care))
    """
    # arragne shapes into shoelace form: [true_base, true_tip, pred_tip, pred_base, true_base]
    LL = tf.gather(LL, [0,1,3,2], axis=2)
    LL = tf.concat((LL, LL[:,:,:1,:]), axis = 2) # :1 to keepdim
    
    sl1 = tf.math.reduce_sum(tf.math.multiply(LL[:,:,:4,0],LL[:,:,1:,1]), axis = 2)
    sl2 = tf.math.reduce_sum(tf.math.multiply(LL[:,:,1:,0],LL[:,:,:4,1]), axis = 2)

    return tf.math.abs(tf.math.subtract(sl1, sl2))


def line_loss(pres_inds, coord_true, coord_pred):    
    """
    loss function that considers lines
    Area of four dots polygon as calculated by shoelace
    """
    m = tf.shape(pres_inds)[0]
    lines_care = [[2,3],[5,6],[7,8],[9,10],[11,12]]# pairs of indices
    n_lines = len(lines_care)
    #line loss pres: to be multiplied with output of shoelace
    pres_list = []
    #line loss coords: input of shoelace, 3rd dim: [true_b, true_t, pred_b, pred_t]. 
    # 4th dim:[Y, X]
    coords_list = []

    for i, line in enumerate(lines_care):
        curr_pres = tf.math.multiply(pres_inds[:,line[0]], pres_inds[:,line[1]])
        t_coords = tf.gather(coord_true, line, axis = 1)
        p_coords = tf.gather(coord_pred, line, axis = 1)
        curr_coords = tf.concat((t_coords, p_coords), axis = 1)
        pres_list.append(curr_pres)
        coords_list.append(curr_coords)
    
    LL_pres = tf.stack(pres_list, axis=1)
    LL_coords = tf.stack(coords_list, axis=1)    
    
    SL = shoelace(LL_coords)
    line_cost = tf.math.reduce_mean(tf.math.multiply(LL_pres, SL))
    return line_cost    

def FP_loss(y_true, y_pred):
    pres_true = y_true[:,:,0]
    coord_true = y_true[:,:,1:]
    pred = tf.reshape(y_pred, (-1, LBL_NUM, LBL_W))

    pres_pred = pred[:,:,0]
    coord_pred = pred[:,:,1:]

    pres_cost = tf.nn.sigmoid_cross_entropy_with_logits(labels = pres_true, logits = pres_pred)
    coord_cost = tf.norm(tf.multiply(coord_pred - coord_true,[1,3]), ord = 2, axis=2)

    pres_not_ignore = tf.math.sign(pres_true + 1)
    pres_inds = tf.nn.relu(pres_true)

    RealPresCost = tf.nn.relu(tf.math.reduce_mean(tf.math.multiply(pres_not_ignore, pres_cost)) - 0.05)
    RealCoordCost = tf.math.reduce_mean(tf.math.multiply(pres_inds, coord_cost))#/pi_sum

    #line_cost = line_loss(pres_inds, coord_true, coord_pred)
    cost = RealPresCost + RealCoordCost# + line_cost*SL_WEIGHT
    
    return cost

def random_color_gen():
    # Generates a random color in hex format
    color = "#"+''.join([random.choice('6789ABCDEF') for j in range(6)])
    #emmitted lower values to make color brighter
    return color

def procure_output(X_test, Y_test, y_pred, out_folder):
    r = [1,1]
    r_size = 2
    out = open(out_folder + 'output.txt', 'w')

    for i in range(X_test.shape[0]):
        img = Image.fromarray((X_test[i,:,:,0] * 255.).astype(np.uint8), 'L')
        img = img.resize((300*r_size, 100*r_size))
        img = img.convert('RGB')
        
        draw = ImageDraw.Draw(img)
        lbl = (Y_test[i,:,:] + [0, 0.5, 0.5]) * [1,100,300] * [1, r_size, r_size]
        pred = (y_pred[i,:].reshape(-1, 3) + [0, 0.5, 0.5]) * [1,100, 300] * [1, r_size, r_size]

        # swap y and x to be x, y
        lbl = lbl[:,[0,2,1]]
        pred = pred[:,[0,2,1]]
        pred[:,0] = tf.math.sigmoid(pred[:,0])
          
        random.seed(0)
        for d in range(lbl.shape[0]):
            color = random_color_gen()
            toWrite = list(map(str, np.concatenate(([i], lbl[d], pred[d]), axis = 0)))
            out.write('\t'.join(toWrite) + '\n')
            if lbl[d,0] == 1:
                start = tuple(lbl[d,1:])
                s0 = tuple(lbl[d,1:] - r)
                s1 = tuple(lbl[d,1:] + r)                
                draw.rectangle([s0, s1], fill = color)
                if pred[d,0] > 0.5:
                    end = tuple(pred[d,1:])                
                    draw.line([start, end], fill = color)
                else:
                    print('false negative!')
                    draw.rectangle([s0, s1], fill = '#FF0000')
            elif pred[d,0] > 0.5:
                s0 = tuple(pred[d,1:] - r)
                s1 = tuple(pred[d,1:] + r) 
                if lbl[d,0] == 0:
                    print('false positive!')
                    draw.ellipse([s0, s1], fill = '#FF0000')
                elif lbl[d,0] == -1:
                    print('noice!')
                    draw.ellipse([s0, s1], outline = color)
        img.save(out_folder + str(i) + '.bmp', 'BMP')
    out.close()

In [0]:
!cp "/content/drive/My Drive/FishPose/Inputs_20200604.zip" "./"
!unzip Inputs_20200604.zip

In [0]:
train_f = 'Inputs_20200604/'
test_f = train_f + 'test_set/'

X_train, Y_train = load_dataset(train_f)
X_test, Y_test = load_dataset(test_f)

print('X_train, Y_train, X_test, Y_test shapes: ', X_train.shape, Y_train.shape, X_test.shape, Y_test.shape)

X_train, Y_train, X_test, Y_test shapes:  (3332, 100, 300, 1) (3332, 13, 3) (24, 100, 300, 1) (24, 13, 3)


In [0]:
model = FP_base()
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    0.001, decay_steps = 200, decay_rate = 0.94)
opt = tf.keras.optimizers.Adam(learning_rate = lr_schedule)
model.compile(optimizer=opt, loss=FP_loss, metrics = [FP_loss_Pres, FP_loss_Coord])
model.build(input_shape= (None, 100,300,1))
#model.summary()

In [0]:
history = model.fit(X_train, Y_train, batch_size=64, epochs=100, validation_data=(X_test, Y_test))

Epoch 1/100
53/53 [==============================] - 2s 29ms/step - loss: 0.2442 - FP_loss_Pres: 0.1149 - FP_loss_Coord: 0.1732 - val_loss: 0.2377 - val_FP_loss_Pres: 0.1323 - val_FP_loss_Coord: 0.1554
Epoch 2/100
53/53 [==============================] - 1s 23ms/step - loss: 0.0948 - FP_loss_Pres: 0.0482 - FP_loss_Coord: 0.0896 - val_loss: 0.1931 - val_FP_loss_Pres: 0.1294 - val_FP_loss_Coord: 0.1137
Epoch 3/100
53/53 [==============================] - 1s 23ms/step - loss: 0.0740 - FP_loss_Pres: 0.0445 - FP_loss_Coord: 0.0716 - val_loss: 0.1520 - val_FP_loss_Pres: 0.1295 - val_FP_loss_Coord: 0.0725
Epoch 4/100
53/53 [==============================] - 1s 23ms/step - loss: 0.0638 - FP_loss_Pres: 0.0447 - FP_loss_Coord: 0.0613 - val_loss: 0.2046 - val_FP_loss_Pres: 0.1259 - val_FP_loss_Coord: 0.1287
Epoch 5/100
53/53 [==============================] - 1s 23ms/step - loss: 0.0662 - FP_loss_Pres: 0.0464 - FP_loss_Coord: 0.0616 - val_loss: 0.1487 - val_FP_loss_Pres: 0.1272 - val_FP_loss_Coor

In [0]:
y_pred = model(X_test)
procure_output(X_test, Y_test, np.array(y_pred), '/content/drive/My Drive/FishPose/output/')


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

false positive!
false negative!
false negative!
false negative!
false negative!
false negative!
false negative!
false negative!
false negative!
false negative!
false negative!
false negative!
false negative!


In [0]:
def output_history(his, outFolder = '/content/drive/My Drive/FishPose/output/'):
    hisDict = his.history
    headers = []
    all_data = []
    for thing in hisDict:
        headers.append(thing)
        all_data.append(hisDict[thing])
    
    Data = np.array(all_data).T
    np.savetxt(outFolder + 'metrics.txt',Data, fmt='%f', delimiter='\t', header = '\t'.join(headers))

output_history(history)

In [0]:
model.save('/content/drive/My Drive/FishPose/output/FishPose_base') 

INFO:tensorflow:Assets written to: /content/drive/My Drive/FishPose/output/FishPose_base/assets
